In [ ]:
# importing all modules
from pymol import cmd
import py3Dmol
import pdbfixer
from openbabel import pybel
from vina import Vina
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, Draw
import sys, os, random
import numpy as np
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import PCA

In [ ]:
cmd.fetch(code='3e64',type='pdb1')
cmd.select(name='prot',selection='polymer.protein')
cmd.select(name='ligand',selection='organic')
cmd.save(filename='3e64_clean.pdb',format='pdb',selection='prot')
cmd.save(filename='3e64_ligand.mol2',format='mol2',selection='ligand')
cmd.delete('all')

In [ ]:
# py3Dmol 
view = py3Dmol.view()
view.removeAllModels()
view.setViewStyle({'style':'outline','color':'black','width':0.1})

view.addModel(open('3e64_clean.pdb','r').read(),format='pdb')
Prot=view.getModel()
Prot.setStyle({'cartoon':{'arrows':True, 'tubes':True, 'style':'oval', 'color':'white'}})
view.addSurface(py3Dmol.VDW,{'opacity':0.6,'color':'white'})

view.addModel(open('3e64_ligand.mol2','r').read(),format='mol2')
ref_m = view.getModel()
ref_m.setStyle({},{'stick':{'colorscheme':'greenCarbon','radius':0.2}})

view.zoomTo()
view.show()

In [ ]:
# prepare the protein structure for docking
from pdbfixer import PDBFixer
from openmm.app import PDBFile

fix = PDBFixer(filename='3e64_clean.pdb')
# find missing residues
fix.findMissingResidues()
# find and replace nonstandard residues
fix.findNonstandardResidues()
fix.replaceNonstandardResidues()
# remove the garbage
fix.removeHeterogens(True)
# find and add missing atoms
fix.findMissingAtoms()
fix.addMissingAtoms()
# add hydrogens at pH=7.4
fix.addMissingHydrogens(7.4)
# write an output fine
PDBFile.writeFile(fix.topology, fix.positions, open('3e64_clean_H.pdb', 'w'))

In [ ]:
# prepare the receptor for docking
! /Users/philipporekhov/soft/ADFRsuite_x86_64Darwin_1.0/bin/prepare_receptor -v -r {'3e64_clean_H.pdb'} -o {'3e64_clean_H.pdbqt'}

In [ ]:
# define the size of docking box (+- 5 A from the edges of the native ligand)
cmd.load(filename='3e64_clean_H.pdb',format='pdb',object='prot')
cmd.load(filename='3e64_ligand.mol2',format='mol2',object='lig')

([minX, minY, minZ],[maxX, maxY, maxZ]) = cmd.get_extent('lig')

minX = minX - 5.0
minY = minY - 5.0
minZ = minZ - 5.0
maxX = maxX + 5.0
maxY = maxY + 5.0
maxZ = maxZ + 5.0
    
SizeX = maxX - minX
SizeY = maxY - minY
SizeZ = maxZ - minZ
CenterX =  (maxX + minX)/2
CenterY =  (maxY + minY)/2
CenterZ =  (maxZ + minZ)/2

cmd.delete('all')

center = {'center_x':CenterX,'center_y': CenterY, 'center_z': CenterZ}
size = {'size_x':SizeX,'size_y': SizeY,'size_z': SizeZ}
print(center,'\n',size)

In [ ]:
# read ligands found using the pharmacophore model
for i, mol in enumerate(pybel.readfile("sdf", "query_results.sdf")):
    mol.make3D()
    mol.write(format='pdbqt', filename='mol_' + str(i) +'.pdbqt', overwrite=True)

In [ ]:
mol = next(pybel.readfile("mol2", "3e64_ligand.mol2"))
mol.make3D()
mol.write(format='pdbqt', filename='3e64_ligand.pdbqt', overwrite=True)

In [ ]:
# initialize and run vina docking for the native ligand
v = Vina(sf_name='vina')
v.set_receptor('3e64_clean_H.pdbqt')
v.compute_vina_maps(center=[center['center_x'], center['center_y'], center['center_z']], 
                    box_size=[size['size_x'], size['size_y'], size['size_z']])


v.set_ligand_from_file('3e64_ligand.pdbqt')
v.dock(exhaustiveness=10, n_poses=1)
v.write_poses('3e64_ligand_vina_out.pdbqt', n_poses=1, overwrite=True)

In [ ]:
# print the score
print(v.score()[0])
# write out the dicking pose
mol = next(pybel.readfile("pdbqt", "3e64_ligand_vina_out.pdbqt"))
mol.write(format='mol2', filename='3e64_ligand_vina_out.mol2', overwrite=True)

In [ ]:
# show it
view = py3Dmol.view()
view.removeAllModels()
view.setViewStyle({'style':'outline','color':'black','width':0.1})

view.addModel(open('3e64_clean_H.pdb','r').read(), format='pdb')
Prot=view.getModel()
Prot.setStyle({'cartoon':{'arrows':True, 'tubes':True, 'style':'oval', 'color':'white'}})
view.addSurface(py3Dmol.VDW,{'opacity':0.6,'color':'white'})


view.addModel(open('3e64_ligand.mol2','r').read(),format='mol2')
ref_m = view.getModel()
ref_m.setStyle({},{'stick':{'colorscheme':'magentaCarbon','radius':0.2}})

view.addModel(open('3e64_ligand_vina_out.mol2','r').read(),format='mol2')
ref_m = view.getModel()
ref_m.setStyle({},{'stick':{'colorscheme':'cyanCarbon','radius':0.2}})

view.zoomTo()
view.show()

In [ ]:
# run vina docking for all ligands
v = Vina(sf_name='vina')
v.set_receptor('3e64_clean_H.pdbqt')
v.compute_vina_maps(center=[center['center_x'], center['center_y'], center['center_z']], 
                    box_size=[size['size_x'], size['size_y'], size['size_z']])

ligands = ['mol_' + str(i) + '.pdbqt' for i in range(50)]
scores = []

for i, ligand in enumerate(ligands):
    v.set_ligand_from_file(ligand)
    v.dock(exhaustiveness=10, n_poses=1)
    v.write_poses('mol_' + str(i) + '_vina_out.pdbqt', n_poses=1, overwrite=True)
    scores.append(v.score()[0])

In [ ]:
best_score = scores.index(min(scores))
print(best_score)

In [ ]:
# generate fingerpring for the native ligand
mol = next(pybel.readfile("mol2", "3e64_ligand.mol2"))
ref_lig_smi = mol.write().split('\t')[0]
ref_mol = Chem.MolFromSmiles(ref_lig_smi)

ref_fps = AllChem.GetMorganFingerprintAsBitVect(ref_mol,2)
print(ref_fps.ToBitString())
ref_mol

In [ ]:
# calculate the similarity metrics (Tanimoto)
ligs = Chem.SDMolSupplier('query_results.sdf')
ligs_fps = [AllChem.GetMorganFingerprintAsBitVect(i, 2) for i in ligs]
similarity = [DataStructs.FingerprintSimilarity(ref_fps, i) for i in ligs_fps]

print(max(similarity), '\n', similarity[best_score])

In [ ]:
print(Chem.MolToSmiles(ligs[best_score]))
print(ref_lig_smi)

In [ ]:
# read  the dataset with water solubility data and smiles
import pandas as pd
esol_data = pd.read_csv('alogps_2_1_logs_training_.csv')
PandasTools.AddMoleculeColumnToFrame(esol_data, smilesCol='SMILES')

# generate fingerprints for all compounds in the table
radius=3
nBits=1024
ECFP6 = [AllChem.GetMorganFingerprintAsBitVect(x,radius=radius, nBits=nBits) for x in esol_data['ROMol']]
ecfp6_name = [f'Bit_{i}' for i in range(nBits)]
ecfp6_bits = [list(l) for l in ECFP6]
df_morgan = pd.DataFrame(ecfp6_bits, index = esol_data.SMILES, columns=ecfp6_name)
df_morgan

In [ ]:
# make PCA for fingerprints
from sklearn.decomposition import PCA

pca = PCA(n_components=10, random_state=0)
esol_pca = pca.fit_transform(df_morgan)

In [ ]:
# split the dateset to training and test sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(esol_pca,
                                                    esol_data['Water solubility'],
                                                    test_size=0.30,
                                                    random_state=1234)

In [ ]:
# train gradient boosting regression model and predict
# solubility for training and test sets
from sklearn.ensemble import GradientBoostingRegressor
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, mean_squared_error

reg = GradientBoostingRegressor(random_state=0)
reg.fit(X_train, y_train)
pred = reg.predict(X_train)
print(r2_score(y_train, pred))
print(mean_squared_error(y_train, pred))
plt.plot(pred, y_train, lw = 0, marker = 'o')
plt.show()

pred = reg.predict(X_test)
print(r2_score(y_test, pred))
print(mean_squared_error(y_test, pred))
plt.plot(pred, y_test, lw = 0, marker = 'o')
plt.show()

In [ ]:
# predict solubility for the docked ligands
ligs = Chem.SDMolSupplier('query_results.sdf')
ligs_fp = [AllChem.GetMorganFingerprintAsBitVect(x,radius=radius, nBits=nBits) for x in ligs]
ligs_ecfp6_bits = [list(l) for l in ligs_fp]
ligs_df_morgan = pd.DataFrame(ligs_ecfp6_bits, index = np.arange(0, 50), columns=ecfp6_name)
ligs_pca = pca.fit_transform(ligs_df_morgan)
ligs_pred = reg.predict(ligs_pca)
# solubility for the best scored ligand, in M/L
ligs_pred[best_score]